In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
                    .appName("sparkml")\
                    .getOrCreate()
spark.sparkContext.setLogLevel("WARN") 

In [3]:
df = pd.read_csv("training.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227846 entries, 0 to 227845
Data columns (total 30 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      227846 non-null  float64
 1   V2      227846 non-null  float64
 2   V3      227846 non-null  float64
 3   V4      227846 non-null  float64
 4   V5      227846 non-null  float64
 5   V6      227846 non-null  float64
 6   V7      227846 non-null  float64
 7   V8      227846 non-null  float64
 8   V9      227846 non-null  float64
 9   V10     227846 non-null  float64
 10  V11     227846 non-null  float64
 11  V12     227846 non-null  float64
 12  V13     227846 non-null  float64
 13  V14     227846 non-null  float64
 14  V15     227846 non-null  float64
 15  V16     227846 non-null  float64
 16  V17     227846 non-null  float64
 17  V18     227846 non-null  float64
 18  V19     227846 non-null  float64
 19  V20     227846 non-null  float64
 20  V21     227846 non-null  float64
 21  V22     22

In [5]:
df.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,...,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000
mean,0.001826,0.000326,0.001060,-0.001550,-0.002031,-0.001750,-0.000212,-0.000294,-0.001407,-0.000029,...,-0.000275,0.000411,0.000917,0.000615,-0.000063,0.000149,0.000449,0.000418,88.184738,0.001773
std,1.954579,1.644444,1.516323,1.413616,1.382919,1.332016,1.240424,1.190580,1.098079,1.089207,...,0.730956,0.725130,0.625545,0.605342,0.520599,0.482131,0.403203,0.328762,248.853646,0.042071
min,-56.407510,-72.715728,-48.325589,-5.683171,-113.743307,-26.160506,-41.506796,-50.943369,-13.434066,-24.403185,...,-22.889347,-10.933144,-44.807735,-2.822384,-10.295397,-2.534330,-22.565679,-15.430084,0.000000,0.000000
25%,-0.920080,-0.597339,-0.887821,-0.848916,-0.693341,-0.768616,-0.554195,-0.208444,-0.643000,-0.534480,...,-0.228480,-0.542160,-0.161698,-0.354217,-0.317246,-0.326968,-0.070767,-0.052728,5.510000,0.000000
50%,0.020316,0.065496,0.180700,-0.020630,-0.056563,-0.275346,0.040038,0.022505,-0.052269,-0.091966,...,-0.029516,0.006549,-0.011290,0.041618,0.016754,-0.051985,0.001381,0.011330,21.980000,0.000000
75%,1.315753,0.802846,1.026490,0.740214,0.609889,0.393989,0.570629,0.326284,0.595955,0.455591,...,0.186399,0.529791,0.147771,0.439527,0.350562,0.240824,0.091382,0.078410,77.170000,0.000000
max,2.454930,21.467203,9.382558,16.715537,34.801666,73.301626,120.589494,20.007208,15.594995,23.745136,...,27.202839,8.361985,22.528412,4.584549,7.519589,3.517346,31.612198,33.847808,25691.160000,1.000000


In [6]:
df['Class'].value_counts()

Class
0    227442
1       404
Name: count, dtype: int64

In [7]:
train = spark.createDataFrame(df) 

In [8]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [9]:
assembler = VectorAssembler(inputCols=df.columns[:-1].tolist(),outputCol="features")

In [10]:
# Train a GBT model.
gbt = GBTClassifier(labelCol="Class", featuresCol="features", maxIter=100)

In [11]:
(xtrain, xtest) = train.randomSplit([0.8, 0.2])

In [12]:
pipeline = Pipeline(stages=[assembler, gbt])

In [13]:
gbt = pipeline.fit(xtrain)

In [14]:
predictions = gbt.transform(xtest)

In [15]:
evaluator = BinaryClassificationEvaluator(
    labelCol="Class", rawPredictionCol="prediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print(f"Test AUC = {auc}")
print(gbt)

Test AUC = 0.9011793497896939
PipelineModel_eecfa59b97d3


In [16]:
# Make sure to move this to the /src folder when done.
gbt.save("/src/gbt_model.model")

Py4JJavaError: An error occurred while calling o317.save.
: java.io.IOException: Path /src/gbt_model.model already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.super$save(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$4(Pipeline.scala:344)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent(events.scala:174)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent$(events.scala:169)
	at org.apache.spark.ml.util.Instrumentation.withSaveInstanceEvent(Instrumentation.scala:42)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3$adapted(P